In [39]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [61]:
# Load your CSV file
csv_file_path = 'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\hl_genes_with_go_terms_lists.csv'
df = pd.read_csv(csv_file_path)

def delete_sign_from_column(dataframe, column_name, sign_to_remove):
    dataframe[column_name] = dataframe[column_name].str.replace(sign_to_remove, "")
    return dataframe

def split_to_list(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.split(', ')
    return dataframe

# remove all ' signs from the go terms
for column in ['Cellular Component', 'Biological Process', 'Molecular Function']:
    for sign in ["'", "[", "]"]:
        df = delete_sign_from_column(df, column, sign)
        
# split the go terms to lists
for column in ['Cellular Component', 'Biological Process', 'Molecular Function']:
    df = split_to_list(df, column)

df.head()

# Print 

,Gene,Cellular Component,Biological Process,Molecular Function
0,ATP6V1B2,"[apical plasma membrane, clathrin-coated vesic...","[ATP metabolic process, proton transmembrane t...","[ATP binding, proton transmembrane transporter..."
1,KITLG,"[cytoplasm, cytoskeleton, extracellular region...","[cell adhesion, ectopic germ cell programmed c...","[cytokine activity, growth factor activity, st..."
2,TCOF1,"[cytosol, fibrillar center, nucleolus, nucleop...","[neural crest cell development, neural crest f...","[protein heterodimerization activity, protein-..."
3,PTPRQ,"[membrane, receptor complex]","[dephosphorylation, regulation of fat cell dif...",[protein tyrosine phosphatase activity]
4,COL2A1,"[basement membrane, collagen type II trimer, c...","[anterior head development, cartilage condensa...",[extracellular matrix structural constituent c...


In [ ]:
# Load a pre-trained sentence transformer model
model_name = 'bert-base-nli-mean-tokens'  # You can choose a different model
bio_model = "dmis-lab/biobert-v1.1"
model = SentenceTransformer(model_name)

# Function to extract embeddings for a list of terms
def get_embeddings(terms):
    if not terms:
        return None
    # Join the terms into a single string
    text = ', '.join(terms)
    print(text)
    # Get the embeddings for the combined string
    embedding = model.encode(text)
    return embedding

def get_averaged_embeddings(terms):
    if not terms:
        return None
    # Join the terms into a single string
    text = ' '.join(terms)
    # Get the embeddings for the combined string
    embedding = model.encode(text)
    averaged_embedding = np.mean(embedding, axis=0)
    return averaged_embedding

# Apply the function to each row in the DataFrame
df['Cellular_Component_Embedding'] = df['Cellular Component'].apply(lambda x: get_embeddings(str(x).split(', ')))
df['Biological_Process_Embedding'] = df['Biological Process'].apply(lambda x: get_embeddings(str(x).split(', ')))
df['Molecular_Function_Embedding'] = df['Molecular Function'].apply(lambda x: get_embeddings(str(x).split(', ')))

df.head()

In [33]:
# Save the DataFrame to a CSV file
csv_file_path = f'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\go terms\\hl_genes_with_go_terms_embeddings_29.1.csv'
df.to_csv(csv_file_path, index=False)

In [38]:
from sklearn.cluster import KMeans
import numpy as np

# Load the DataFrame with embeddings
df = pd.read_csv(f'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\go terms\\hl_genes_with_go_terms_embeddings_lists_29.1.csv')

# Specify the number of clusters
num_clusters = 5  # You can adjust this number based on your data

# Define a function to perform clustering for a specific category
def perform_clustering(embedding_column, num_clusters):
    # Extract the embeddings for the specific category as a NumPy array
    embeddings = np.array(df[embedding_column].to_list())

    # Perform K-means clustering for the specific category
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    df[f'{embedding_column}_Cluster_Labels'] = kmeans.fit_predict(embeddings)

# Perform clustering for each category
for category in ['Cellular_Component']:
    embedding_column = f'{category}_Embedding'
    perform_clustering(embedding_column, num_clusters)

# Display the clustering results for each category
print(df[['Gene', 'Cellular_Component_Cluster_Labels', 'Biological_Process_Cluster_Labels', 'Molecular_Function_Cluster_Labels']])

# Save the DataFrame with cluster labels for each category to a new CSV file
clusters_path = f'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\go terms\\hl_genes_with_go_terms_embeddings_clusters_29.1.csv'
df.to_csv('gene_clusters_separate_categories.csv', index=False)

ValueError: could not convert string to float: '[-6.20042741e-01  2.80665278e-01  1.23496294e+00  5.39861917e-01\n  1.16935945e+00 -1.47770000e+00 -4.15638298e-01 -2.98210174e-01\n  4.29672480e-01 -3.50228786e-01 -4.27606672e-01  4.04825598e-01\n -1.98288262e-01  9.61116552e-01  1.08708751e+00  2.59018749e-01\n -5.82380474e-01  5.96343398e-01  6.36185288e-01 -1.78393796e-01\n -4.19351518e-01 -2.70610303e-01  8.66798103e-01  4.00193095e-01\n  1.30106795e+00  1.49848625e-01 -1.99390277e-01 -7.58935586e-02\n  2.46396810e-01 -5.01226857e-02 -1.36508629e-01  3.42029303e-01\n -1.39853835e-01 -9.63443220e-02 -3.61858636e-01  5.46477377e-01\n -7.15613782e-01 -3.72056067e-02  1.80983394e-01  9.07226931e-03\n -6.02528334e-01 -7.08665669e-01 -1.93411991e-01 -1.17238127e-01\n -1.12165499e+00  1.52819557e-02  3.39499503e-01 -1.79130480e-01\n -1.08564675e+00 -1.16066188e-01 -3.82931620e-01  9.04817700e-01\n -2.04431191e-01 -1.31014645e-01  1.62631527e-01  3.94540757e-01\n -1.88418776e-01 -9.06178772